In [1]:
from boututils.datafile import DataFile
from boutdata.collect import collect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pathlib
import platform
import traceback
import xarray
import xbout
import scipy
import re
import netCDF4 as nc

onedrive_path = onedrive_path = str(os.getcwd()).split("OneDrive")[0] + "OneDrive"
sys.path.append(os.path.join(onedrive_path, r"Project\python-packages\sdtools"))
sys.path.append(os.path.join(onedrive_path, r"Project\python-packages\soledge"))
sys.path.append(os.path.join(onedrive_path, r"Project\python-packages"))


from gridtools.b2_tools import *
from gridtools.utils import *

from hermes3.fluxes import *
from hermes3.case_db import *
from hermes3.load import *
from hermes3.named_selections import *
from hermes3.plotting import *
from hermes3.grid_fields import *
from hermes3.accessors import *
from hermes3.utils import *
from code_comparison.viewer_2d import *
from code_comparison.code_comparison import *
from code_comparison.solps_pp import *

from gridtools.solps_python_scripts.read_b2fgmtry import *


%load_ext autoreload
%autoreload 2
%matplotlib inline

print("Done")


st40_phys_viewer can not be loaded from outside Tokamak Energy!

Done


In [2]:
path=os.path.join(r"C:\Users\mikek\OneDrive\Project\collab\tech\compare_data\soledge_cases\ST40_wallStd_bra_nocore_D_2e19")

In [7]:
self = SOLEDGEcase(path)

c:\Users\mikek\OneDrive\Project\python-packages\soledge\routines\h5_routines.py:65: FutureWarning: In the future `np.str` will be defined as the corresponding NumPy scalar.
  Data = Data.astype(np.str)


In [12]:
from routines.globals					import DEBUG, KB
from files.load_fluxes_files			import load_fluxes_files

In [9]:
shading = 'gouraud'

Config = self.Config
Zones	= Config.Zones
nZones	= len(Zones)
RefPar = self.RefPar
ions = self.ions
d_only = 0

if(d_only != 0): ions = ions[0:2]

Titles 	= [["$\Gamma_E\ North\ (MW/m^2)$",				"$\Gamma_E\ South\ (MW/m^2)$",	 			"$\Gamma_E\ East\ (MW/m^2)$",  			"$\Gamma_E\ West\ (MW/m^2)$", 
            "$\Gamma_E\ S-N\ (MW/m^2)$", 				"$\Gamma_E\ W-E\ (MW/m^2)$", 				"$\Gamma_E\ S-N+W-E\ (MW/m^2)$", \
            "$\Gamma_n\ North\ *10^{20}\ m^{-2})$", 	"$\Gamma_n\ South\ *10^{20}\ m^{-2})$",	"$\Gamma_n\ East\ *10^{20}\ m^{-2})$",	"$\Gamma_n\ West\ *10^{20}\ m^{-2})$", 
            "$\Gamma_n\ S-N\ (*10^{20}\ m^{-2})$",		"$\Gamma_n\ W-E\ (*10^{20}\ m^{-2})$",		"$\Gamma_n\ S-N+W-E\ (*10^{20}\ m^{-2})$"]]

EFluxFact 	= 1e-6*KB*RefPar.n0*RefPar.c0*RefPar.T0*RefPar.rs0/(2*np.pi*RefPar.R0)
nFluxFact 	= 1e-20*RefPar.n0*RefPar.c0*RefPar.rs0/(2*np.pi*RefPar.R0)
FluxFacts	= [[EFluxFact, EFluxFact, EFluxFact, EFluxFact, EFluxFact, EFluxFact, EFluxFact, nFluxFact, nFluxFact, nFluxFact, nFluxFact, nFluxFact, nFluxFact, nFluxFact]]

iValues		= [[0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2]]
iPars		= [[0, 1, 2, 3,-1,-2,-3, 0, 1, 2, 3,-1,-2,-3]]
PosPlots	= [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13]]

if(len(ions) > 1):
    for iPlasma in range(1,len(ions)):
        Titles.append(Titles[0]) 
        iValues.append(iValues[0]) 
        iPars.append(iPars[0]) 
        FluxFacts.append(FluxFacts[0]) 
        PosPlots.append(PosPlots[0][:]) 											#[:] slice to force real copy
        for i in range(len(PosPlots[-1])): PosPlots[-1][i] += PosPlots[-2][-1] + 1

In [27]:
Fluxes = load_fluxes_files(os.path.join(path, "Results"), nZones=nZones, iFluxes = [iPlasma])

Values = []
for i in range(len(PosPlots[iPlasma])): 
    Values.append([])
    for k in range(nZones):
        if(iValues[iPlasma][i] > -1):
            if(iPars[iPlasma][i] > -1):
                Values[-1].append(Fluxes[0][k].Values[iValues[iPlasma][i]][:,:,iPars[iPlasma][i]])
            elif(iPars[iPlasma][i] == -1):
                Values[-1].append(Fluxes[0][k].Values[iValues[iPlasma][i]][:,:,1]-Fluxes[0][k].Values[iValues[iPlasma][i]][:,:,0])	#S-N
            elif(iPars[iPlasma][i] == -2):
                Values[-1].append(Fluxes[0][k].Values[iValues[iPlasma][i]][:,:,3]-Fluxes[0][k].Values[iValues[iPlasma][i]][:,:,2])	#W-E
            elif(iPars[iPlasma][i] == -3):
                Values[-1].append(Fluxes[0][k].Values[iValues[iPlasma][i]][:,:,1]-Fluxes[0][k].Values[iValues[iPlasma][i]][:,:,0]+Fluxes[0][k].Values[iValues[iPlasma][i]][:,:,3]-Fluxes[0][k].Values[iValues[iPlasma][i]][:,:,2]) #S-N+W-E



load_fluxes_files: Loading from  C:\Users\mikek\OneDrive\Project\collab\tech\compare_data\soledge_cases\ST40_wallStd_bra_nocore_D_2e19\Results/fluxes_1


In [45]:
Titles[0]

['$\\Gamma_E\\ North\\ (MW/m^2)$',
 '$\\Gamma_E\\ South\\ (MW/m^2)$',
 '$\\Gamma_E\\ East\\ (MW/m^2)$',
 '$\\Gamma_E\\ West\\ (MW/m^2)$',
 '$\\Gamma_E\\ S-N\\ (MW/m^2)$',
 '$\\Gamma_E\\ W-E\\ (MW/m^2)$',
 '$\\Gamma_E\\ S-N+W-E\\ (MW/m^2)$',
 '$\\Gamma_n\\ North\\ *10^{20}\\ m^{-2})$',
 '$\\Gamma_n\\ South\\ *10^{20}\\ m^{-2})$',
 '$\\Gamma_n\\ East\\ *10^{20}\\ m^{-2})$',
 '$\\Gamma_n\\ West\\ *10^{20}\\ m^{-2})$',
 '$\\Gamma_n\\ S-N\\ (*10^{20}\\ m^{-2})$',
 '$\\Gamma_n\\ W-E\\ (*10^{20}\\ m^{-2})$',
 '$\\Gamma_n\\ S-N+W-E\\ (*10^{20}\\ m^{-2})$']

In [33]:
len(Values[0][0])

9